In [3]:
#many to many
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import matplotlib.pyplot as plt

torch.manual_seed(1)

# Hyperparameters
epochs = 1000
lr = 0.1
input_size = 2
hidden_size = 2
output_size = 2
batch_size = 1
seq_len = 20
num_layers = 1

# Data
inputs = torch.Tensor([[random.random() for _ in range(input_size)] for _ in range(seq_len)])
targets = torch.LongTensor([random.randint(0, 1) for _ in range(seq_len)])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model
# many to many LSTM

class LSTMmany(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMmany, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstmencoder = nn.LSTM(input_size, hidden_size, num_layers)
        self.lstmdecoder = nn.LSTM(hidden_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))
    
    def forward(self, input):
        output, self.hidden = self.lstmencoder(input.view(len(input), batch_size, -1), self.hidden)
        output, self.hidden = self.lstmdecoder(output, self.hidden)
        output = self.linear(output.view(len(input), -1))
        return output
    
model = LSTMmany(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train
for epoch in range(epochs):
    model.zero_grad()
    model.hidden = model.init_hidden()
    input = torch.Tensor(inputs).to(device)
    target = torch.Tensor(targets).to(device)
    output = model(input)
    loss = criterion(output, target.view(-1).long())
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch: %d, loss: %.4f' % (epoch, loss.item()))

Epoch: 0, loss: 0.8144
Epoch: 100, loss: 0.7845
Epoch: 200, loss: 0.1267
Epoch: 300, loss: 0.2149
Epoch: 400, loss: 0.0147
Epoch: 500, loss: 0.0073
Epoch: 600, loss: 0.0045
Epoch: 700, loss: 0.0031
Epoch: 800, loss: 0.0023
Epoch: 900, loss: 0.0018


In [6]:
class ManyDataset(torch.utils.data.Dataset):

    def __init__(self, X, y, day, input_len=128):
        #the input data is a 1d array, indicate the minute of the day
        self.X = X
        self.y = y
        self.day = day
        self.input_len = input_len

    def __getitem__(self, index):
        #output previous self.input_len minutes data and target value
        #if the there is no enough data in the same day, pad with 0
        #y is the target value of future input_len minutes
        d = self.day[index]
        start = index - self.input_len
        if start < 0:
            start = 0
        if self.day[start] != d:
            while self.day[start] != d:
                start += 1
            #pad with 0 before start
        if index - start < self.input_len:
            x = torch.zeros(self.input_len, self.X.shape[1])
            x[self.input_len - index + start: self.input_len] = self.X[start: index].clone()
        else:
            x = self.X[start: index]

        #generate y
        end = index + self.input_len
        if end >= len(self.X):
            end = len(self.X) - 1
        if self.day[end] != d:
            while self.day[end] != d:
                end -= 1
        #pad with 0 after end
        if end - index < self.input_len:
            y = torch.zeros(self.input_len)
            y[0: end - index] = self.y[index: end].clone()
        else:
            y = self.y[index: end]
            

        return x, y
    def __len__(self):
        return len(self.X)

torch.Size([20])

In [7]:
#KNN
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree, BallTree

# generate random regression data
X = np.random.rand(100, 10)
y = np.random.rand(100, 1)

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# instantiate learning model (k = 3)
knn = KNeighborsRegressor(n_neighbors=3, algorithm='kd_tree')


Mean Absolute Error: 0.2462523340681022
Mean Squared Error: 0.08989224660272291
Root Mean Squared Error: 0.2998203572186567
